# Day 06

## Process input data

In [1]:
import numpy as np

In [2]:
def read_data(filename):
    with open (filename) as fh:
        data = np.asarray([int(line.strip()) for line in fh.readlines()])
    return data

In [3]:
sample_transmission = read_data("sample.txt")
transmission = read_data("input.txt")

In [4]:
len(transmission)

1000

In [5]:
PREAMBLE_SIZE = 25

## Part 1

In [6]:
from numba import cuda

In [7]:
@cuda.jit
def validate_number(transmission, output):
    pos = cuda.grid(1)
    if pos < len(transmission) - PREAMBLE_SIZE:
        preamble = transmission[pos:pos+PREAMBLE_SIZE]
        next_number = transmission[pos+PREAMBLE_SIZE]
        for i in range(PREAMBLE_SIZE):
            for j in range(PREAMBLE_SIZE):
                if i != j and preamble[i] != preamble[j]:
                    if preamble[i] + preamble[j] == next_number:
                        output[pos] = 1

In [8]:
threadsperblock = 128
blockspergrid = (len(transmission) - PREAMBLE_SIZE + (threadsperblock - 1)) // threadsperblock

In [21]:
output = np.zeros(len(transmission) - PREAMBLE_SIZE)
validate_number[blockspergrid, threadsperblock](transmission, output)
transmission[np.where(output == 0)[0][0] + PREAMBLE_SIZE]

1309761972

## Part 2

In [17]:
TARGET_NUMBER = 1309761972

In [18]:
@cuda.jit
def contiguous_search(transmission, target, output):
    pos = cuda.grid(1)
    if pos < len(transmission):
        acc = transmission[pos]
        counter = 1
        while acc < target and counter + pos < len(transmission):
            acc = 0
            n_min = 0
            n_max = 0
            for n in transmission[pos:pos+counter]:
                if n_min == 0 or n < n_min:
                    n_min = n
                if n_max == 0 or n > n_max:
                    n_max = n
                acc += n
            counter += 1
            if acc == TARGET_NUMBER:
                output[pos] = n_min + n_max

In [19]:
threadsperblock = 128
blockspergrid = (len(transmission) + (threadsperblock - 1)) // threadsperblock

In [20]:
output = np.zeros(len(transmission))
contiguous_search[blockspergrid, threadsperblock](transmission, TARGET_NUMBER, output)
int(output[np.where(output != 0)[0][0]])

177989832